In [44]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

import boston_valuation as val

Price in Dollars: 827000.0
Upper: 997000.0
Lower: 685000.0
Confidence: 68


In [18]:
dataset = load_boston()
data = pd.DataFrame(data= dataset.data, columns=dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)

log_prices = np.log(dataset.target)
target = pd.DataFrame(log_prices, columns=['PRICE'])
features

,CRIM,ZN,CHAS,NOX,RM,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,0.0,0.538,6.575,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,0.0,0.469,6.421,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,0.0,0.469,7.185,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,0.0,0.458,6.998,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,0.0,0.458,7.147,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,0.0,0.573,6.593,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,0.0,0.573,6.120,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,0.0,0.573,6.976,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,0.0,0.573,6.794,2.3889,1.0,273.0,21.0,393.45,6.48


In [19]:
property_stats = np.ndarray(shape=(1,11))
property_stats = features.mean().values.reshape(1, 11)

In [20]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

RM_IDX = 4
PTRATIO_IDX = 8
CHAS_IDX = 2

MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [21]:
def get_log_estimate(nr_rooms, students_per_classroom, next_to_river=False, high_confidence=True):
   
    # configure property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
    
    # Make prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    # calc range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
        
    return log_estimate, upper_bound, lower_bound, interval

In [22]:
np.median(dataset.target)

21.2

In [23]:
ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(dataset.target)

log_est, upper, lower, conf = get_log_estimate(9, students_per_classroom=15, next_to_river=False, high_confidence=False)

dollar_est = np.e ** log_est * 1000 * SCALE_FACTOR
dollar_high = np.e ** upper * 1000 * SCALE_FACTOR
dollar_low = np.e ** lower * 1000 * SCALE_FACTOR

rounded_est = np.around(dollar_est, -3)
rounded_high = np.around(dollar_high, -3)
rounded_low = np.around(dollar_low, -3)

print(f'Price in Dollars: {rounded_est}')
print(f'Upper: {rounded_high}')
print(f'Lower: {rounded_low}')
print(f'Confidence: {conf}')

Price in Dollars: 827000.0
Upper: 997000.0
Lower: 685000.0
Confidence: 68%


In [42]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    
    """Estimate the price of a property in Boston.
    
    Keyword arguments:
    rm -- number of rooms in the property.
    ptratio -- number of students per teacher in the classroom for the school in the area.
    chas -- True if the property is next to the river, False otherwise.
    large_range -- True for a 95% prediction interval, False for a 68% interval.
    
    """
    
    
    if rm < 1 or ptratio < 1:
        print('That is unrealistic. Try again.')
        return
    
    log_est, upper, lower, conf = get_log_estimate(rm, ptratio, chas, large_range)
    
    dollar_est = np.e ** log_est * 1000 * SCALE_FACTOR
    dollar_high = np.e ** upper * 1000 * SCALE_FACTOR
    dollar_low = np.e ** lower * 1000 * SCALE_FACTOR

    rounded_est = np.around(dollar_est, -3)
    rounded_high = np.around(dollar_high, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'Price in Dollars: {rounded_est}')
    print(f'Upper: {rounded_high}')
    print(f'Lower: {rounded_low}')
    print(f'Confidence: {conf}%')

In [43]:
get_dollar_estimate(2, 30, chas=True)

Price in Dollars: 278000.0
Upper: 404000.0
Lower: 191000.0
Confidence: 95%


In [45]:
val.get_dollar_estimate(2, 30, chas=True)

Price in Dollars: 278000.0
Upper: 404000.0
Lower: 191000.0
Confidence: 95%
